In [ ]:
import time


In [32]:
import requests

In [ ]:
ACCESS_TOKEN =  #тут был токен
GROUP_SHORT_NAME = 'doublebubbletea'
def get_group_id(group_name):
    url = 'https://api.vk.com/method/utils.resolveScreenName'
    params = {
        'screen_name': group_name,
        'access_token': ACCESS_TOKEN,
        'v': '5.199'
    }
    r = requests.get(url, params=params).json()
    return -r['response']['object_id']

In [37]:
def get_all_posts(owner_id):
    all_posts = []
    offset = 0
    while True:
        url = 'https://api.vk.com/method/wall.get'
        params = {
            'owner_id': owner_id,
            'count': 100,
            'offset': offset,
            'access_token': ACCESS_TOKEN,
            'v': '5.199'
        }
        r = requests.get(url, params=params).json()
        items = r.get('response', {}).get('items', [])
        if not items:
            break
        all_posts.extend(items)
        offset += 100
        time.sleep(0.33)
    return all_posts


In [38]:
group_id = get_group_id(GROUP_SHORT_NAME)
posts = get_all_posts(group_id)

In [40]:
def build_dataframe(posts):
    data = []
    for post in posts:
        text = post.get('text', '').strip().replace('\n', ' ')
        likes = post.get('likes', {}).get('count', 0)
        views = post.get('views', {}).get('count', 0)
        date = pd.to_datetime(post.get('date', 0), unit='s')  
        data.append({'date': date, 'text': text, 'likes': likes, 'views': views})
    df = pd.DataFrame(data)
    return df

In [41]:
df = build_dataframe(posts)

In [43]:
df.to_csv('vk.csv')